## Combining Filters

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

factorと同様にfilterも組み合わせることができます。filterの組み合わせは `&` と `|` 演算子を用います。例として、30日間の売買代金の平均の上位10%と終値が20ドルを超える条件を組み合わせます。前者の条件は `AverageDollarVolume` クラスの `percentile_between` メソッドを用います。`percentile_between` メソッドの戻り値は `Filter` を返します。

In [2]:
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(90, 100)

次に終値が20ドルを超える銘柄の filter を作成します。

In [3]:
latest_close = USEquityPricing.close.latest
above_20 = latest_close > 20

`high_dollar_volume` と `above_20` の2つの filter を `&` 演算子で組み合わせます。

In [4]:
tradeable_filter = high_dollar_volume & above_20

このfilterは、両方が `True` の場合に `True` と評価されます。それ以外の場合は、`False` と評価されます。  
作成した `tradeable_filter` を piplineのキーワード引数 `screen` に渡します。

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20

    tradeable_filter = high_dollar_volume & above_20

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=tradeable_filter
    )

pipline を実行すると 700銘柄ほどに絞り込まれました。

In [6]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result.head()

Number of securities that passed the filter: 742


percent_difference
2015-05-05 00:00:00+00:00 Equity(24 [AAPL])             0.016905
                          Equity(62 [ABT])              0.014385
                          Equity(67 [ADSK])            -0.003921
                          Equity(76 [TAP])             -0.008759
                          Equity(114 [ADBE])            0.009499

次のレッスンではfactorとfilterのマスキングを実施します。